# random notebook to interactively explore issues

In [1]:
# Point Python to the path where we have installed the bad and darc packages
import sys
sys.path.insert(0, '/Users/btvincent/git-local/darc-experiments-python')

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import seaborn as sns

import darc
from darc.delayed import models
from darc.designs import BayesianAdaptiveDesignGeneratorDARC, DesignSpaceBuilder
from darc.delayed.designs import Griskevicius2011, Frye, Koffarnus_Bickel, Kirby2009

from darc.data_plotting import plot_delay_without_front_end_delays
from darc.parameter_recovery import simulated_experiment_trial_loop

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

## do things...

In [3]:
true_params = pd.DataFrame.from_dict({'logk': [norm(-4.5, 1).rvs()], 'α': [2]})

D = DesignSpaceBuilder(RA=[10., 20., 30., 40., 50., 60., 70., 80., 90.]).build()
design_thing = BayesianAdaptiveDesignGeneratorDARC(D)
model = models.Hyperbolic(n_particles=5000)
model.θ_true = true_params

In [4]:
fitted_model, summary_stats = simulated_experiment_trial_loop(design_thing, model)

/Users/btvincent/git-local/darc-experiments-python/bad/optimisation.py:153: RuntimeWarning: invalid value encountered in true_divide
  (n_times_sampled+n_times_sampled_iter))
/Users/btvincent/git-local/darc-experiments-python/bad/optimisation.py:226: RuntimeWarning: invalid value encountered in true_divide
  (n_times_sampled + n_times_sampled_iter))


In [5]:
design_thing.all_possible_designs.head()

,RA,DA,PA,RB,DB,PB
0,10.0,0.0,1.0,100.0,0.000694,1.0
1,10.0,0.0,1.0,100.0,0.001389,1.0
2,10.0,0.0,1.0,100.0,0.003472,1.0
3,10.0,0.0,1.0,100.0,0.006944,1.0
4,10.0,0.0,1.0,100.0,0.010417,1.0


In [6]:
design_thing.data.df

,RA,DA,PA,RB,DB,PB,R
0,50.0,0.0,1.0,100.0,90.0,1.0,1
0,50.0,0.0,1.0,100.0,180.0,1.0,1
0,60.0,0.0,1.0,100.0,180.0,1.0,1
0,60.0,0.0,1.0,100.0,240.0,1.0,1
0,40.0,0.0,1.0,100.0,730.0,1.0,1
0,30.0,0.0,1.0,100.0,1460.0,1.0,0
0,70.0,0.0,1.0,100.0,240.0,1.0,1
0,70.0,0.0,1.0,100.0,270.0,1.0,0
0,70.0,0.0,1.0,100.0,240.0,1.0,0
0,40.0,0.0,1.0,100.0,730.0,1.0,1


In [7]:
design_cols = ['RA', 'DA', 'PA', 'RB', 'DB', 'PB']

new = design_thing.data.df.groupby(design_cols).size().reset_index(name='freq')
new['prop'] = design_thing.data.df.groupby(design_cols)['R'].mean().reset_index(name='prop')['prop']

In [8]:
new['prop'] = design_thing.data.df.groupby(design_cols)['R'].mean().reset_index(name='prop')['prop']
new

,RA,DA,PA,RB,DB,PB,freq,prop
0,30.0,0.0,1.0,100.0,1460.0,1.0,2,0.000000
1,40.0,0.0,1.0,100.0,730.0,1.0,2,1.000000
2,50.0,0.0,1.0,100.0,90.0,1.0,1,1.000000
3,50.0,0.0,1.0,100.0,180.0,1.0,1,1.000000
4,60.0,0.0,1.0,100.0,180.0,1.0,1,1.000000
5,60.0,0.0,1.0,100.0,240.0,1.0,1,1.000000
6,60.0,0.0,1.0,100.0,365.0,1.0,2,1.000000
7,70.0,0.0,1.0,100.0,240.0,1.0,3,0.666667
8,70.0,0.0,1.0,100.0,270.0,1.0,3,0.333333
9,80.0,0.0,1.0,100.0,150.0,1.0,3,1.000000


In [9]:
new['x'] = new['DB']
new['y'] = new['RA']/new['RB']
new = new[['x', 'y', 'freq', 'prop']]
new

,x,y,freq,prop
0,1460.0,0.3,2,0.000000
1,730.0,0.4,2,1.000000
2,90.0,0.5,1,1.000000
3,180.0,0.5,1,1.000000
4,180.0,0.6,1,1.000000
5,240.0,0.6,1,1.000000
6,365.0,0.6,2,1.000000
7,240.0,0.7,3,0.666667
8,270.0,0.7,3,0.333333
9,150.0,0.8,3,1.000000


In [10]:
new['prop'].values * [[1], [1], [1]]

array([[0.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 0.66666667, 0.33333333, 1.        ,
        0.        ],
       [0.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 0.66666667, 0.33333333, 1.        ,
        0.        ],
       [0.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 0.66666667, 0.33333333, 1.        ,
        0.        ]])

In [11]:


# design_thing.data.df['R'] = design_thing.data.df['R'].astype('float64')
# design_thing.data.df['R']

In [12]:
temp = model.θ['logk']

temp

2868   -6.449816
2287   -6.412056
1169   -6.423006
2597   -6.381010
29     -6.497879
4944   -6.402728
4382   -6.492626
2363   -6.432965
2822   -6.487252
2912   -6.416569
3198   -6.413265
2436   -6.450289
4121   -6.463333
1169   -6.423006
4845   -6.398807
4944   -6.402728
2868   -6.449816
1828   -6.370941
1742   -6.410675
1514   -6.430602
474    -6.305559
4016   -6.431958
391    -6.505412
936    -6.497745
3129   -6.439600
2267   -6.286920
434    -6.501271
780    -6.416159
659    -6.350924
2551   -6.335346
          ...   
4548   -6.357586
2740   -6.373814
2287   -6.412056
1169   -6.423006
4845   -6.398807
2141   -6.353392
2788   -6.356393
1133   -6.411323
2774   -6.335318
1978   -6.385830
2597   -6.381010
1617   -6.384237
2363   -6.432965
2822   -6.487252
4845   -6.398807
3578   -6.408874
2171   -6.375722
4944   -6.402728
2363   -6.432965
1133   -6.411323
3092   -6.432070
3092   -6.432070
4253   -6.456977
4926   -6.378498
2788   -6.356393
3092   -6.432070
3142   -6.341224
2363   -6.4329

In [13]:
temp = model.θ['α']

temp

2868    2.798968
2287    2.023446
1169    0.513386
2597    1.403917
29      2.940628
4944    1.892252
4382    5.576500
2363    1.792570
2822    2.957575
2912    3.110856
3198    5.653164
2436    2.201659
4121    2.144318
1169    0.513386
4845    2.077655
4944    1.892252
2868    2.798968
1828    2.161437
1742    2.908720
1514    1.910680
474     3.280466
4016    0.171339
391     2.341202
936     2.185313
3129    2.346190
2267    2.886195
434     3.935374
780     2.326435
659     1.753119
2551    3.105799
          ...   
4548    1.988213
2740    3.310934
2287    2.023446
1169    0.513386
4845    2.077655
2141    2.856679
2788    1.596842
1133    2.138730
2774    1.839764
1978    2.680556
2597    1.403917
1617    1.898984
2363    1.792570
2822    2.957575
4845    2.077655
3578    1.910819
2171    3.726809
4944    1.892252
2363    1.792570
1133    2.138730
3092    0.357274
3092    0.357274
4253    0.834253
4926    3.157888
2788    1.596842
3092    0.357274
3142    2.589525
2363    1.7925